# Installing libraries

Installing HuggingFace Transformers (https://github.com/huggingface/transformers)

In [1]:
!pip install datasets transformers scikit-learn torch pandas evaluate tensorboardX

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.1 MB/s eta 0:00:00


# Dataset processing

Uploading the dataset, splitting the data into train and validation sets

In [2]:
import pandas as pd
from datasets import DatasetDict, Dataset


data = pd.read_json('./pages/unreliable-sources.json', encoding='utf-8')

train_data = data.sample(frac=0.8, random_state=42)
validation_data = data.drop(train_data.index)

dataset = DatasetDict(
    {'train': Dataset.from_pandas(train_data).shuffle(),
     'validation': Dataset.from_pandas(validation_data).shuffle()
     })

In [3]:
dataset

DatasetDict({
    train: Dataset({
        features: ['page_id', 'revision_id', 'title', 'content', 'has_template', '__index_level_0__'],
        num_rows: 2098
    })
    validation: Dataset({
        features: ['page_id', 'revision_id', 'title', 'content', 'has_template', '__index_level_0__'],
        num_rows: 525
    })
})

In [4]:
from datasets import load_dataset
from transformers import AutoModelForSequenceClassification, AutoTokenizer, DataCollatorWithPadding, TrainingArguments, Trainer

def builder(model_name, raw_dataset):
  tokenizer = AutoTokenizer.from_pretrained(model_name)
  data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

  def tokenize_function(sample):
      return tokenizer(sample["content"], truncation=True)

  def load_model():
    return AutoModelForSequenceClassification.from_pretrained(model_name)

  tokenized_datasets = raw_dataset.map(tokenize_function, batched=True)
  tokenized_datasets = tokenized_datasets.remove_columns(["content", "__index_level_0__"])
  tokenized_datasets = tokenized_datasets.rename_column("has_template", "labels")
  tokenized_datasets.set_format("torch")
  return load_model, tokenized_datasets, data_collator, tokenizer

load_model, tokenized_datasets, data_collator, tokenizer = builder("FacebookAI/roberta-large", dataset)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Map:   0%|          | 0/2098 [00:00<?, ? examples/s]

Map:   0%|          | 0/525 [00:00<?, ? examples/s]

In [5]:
from dataclasses import dataclass

@dataclass
class HyperParameters:
    batch_size: int
    number_of_epochs: int
    learning_rate: float
    warm_up_steps: int

In [6]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def compute_metrics(pred):
  labels = pred.label_ids
  preds = pred.predictions.argmax(-1)
  precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="weighted")
  acc = accuracy_score(labels, preds)
  return {
      "accuracy": acc,
      "f1": f1,
      "precision": precision,
      "recall": recall
    }

In [7]:
def get_training_results(model, hyperparameters):
    args = TrainingArguments(
        output_dir=f"./models/",
        evaluation_strategy = "epoch",
        save_strategy="epoch",
        learning_rate=hyperparameters.learning_rate,
        per_device_train_batch_size=hyperparameters.batch_size,
        per_device_eval_batch_size=hyperparameters.batch_size,
        num_train_epochs=hyperparameters.number_of_epochs,
        report_to='tensorboard',
        logging_dir=f'./logs/',
        load_best_model_at_end=True,
        weight_decay=0.01
    )
    trainer = Trainer(
        model=model,
        args=args,
        train_dataset=tokenized_datasets["train"],
        eval_dataset=tokenized_datasets["validation"],
        tokenizer=tokenizer,
        compute_metrics=compute_metrics
    )
    trainer.train()
    trainer.save_model()
    results = trainer.evaluate()
    return results, model

In [8]:
def optimize(combinations, load_model, tokenized_datasets):
  batch_sizes = {hyperparameters.batch_size: hyperparameters for hyperparameters in combinations}
  max_accuracy = 0
  max_hyperparameters = None
  for batch_size in batch_sizes.keys():
    for hyperparameters in combinations:
      if batch_size != hyperparameters.batch_size:
        continue

      model = load_model()
      model.to(device)
      print(f"============== Hyperparameters: {hyperparameters} ==============")
      results, model = get_training_results(model, hyperparameters)
      if results["eval_accuracy"] > max_accuracy:
        max_accuracy = results["eval_accuracy"]
        max_hyperparameters = hyperparameters

      print(f"Validation accuracy: {results}")

  return max_hyperparameters

In [9]:
import torch
from tqdm.auto import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

batch_sizes = [4]
number_of_epochs = [5]
learning_rates = [3e-5]
warmup_steps = [2]

combinations = [
    HyperParameters(batch_size, number_of_epoch, learning_rate, warmup_step)
    for batch_size in batch_sizes
    for number_of_epoch in number_of_epochs
    for learning_rate in learning_rates
    for warmup_step in warmup_steps
]

optimal_hyperparameters = optimize(combinations, load_model, tokenized_datasets)

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at FacebookAI/roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


============== Hyperparameters: HyperParameters(batch_size=4, number_of_epochs=5, learning_rate=3e-05, warm_up_steps=2) ==============


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.726000,0.708776,0.474286,0.305161,0.224947,0.474286
2,0.710500,0.695374,0.525714,0.362290,0.276376,0.525714
3,0.702700,0.691986,0.525714,0.362290,0.276376,0.525714
4,0.704100,0.693622,0.474286,0.305161,0.224947,0.474286
5,0.701500,0.693417,0.474286,0.305161,0.224947,0.474286


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/m

Validation accuracy: {'eval_loss': 0.6919857263565063, 'eval_accuracy': 0.5257142857142857, 'eval_f1': 0.36228999464954526, 'eval_precision': 0.2763755102040816, 'eval_recall': 0.5257142857142857, 'eval_runtime': 13.724, 'eval_samples_per_second': 38.254, 'eval_steps_per_second': 9.618, 'epoch': 5.0}


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
